## MACHINE LEARNING EN PYTHON 

In [1]:
# Importamos las librerías necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings         # librería que evita que las alertas se muestren
warnings.filterwarnings('ignore')

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
#from sklearn import preprocessing
#from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.model_selection import KFold
#from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel  #seleción de variables
#from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


## Validación Cruzada
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV         #optimización de hiper-parámetros con validación cruzada

# visualization
import seaborn as sns

# Valor de semilla común que se utilizará cuando sea necesario
seed = 16
np.random.seed(seed)

In [2]:
#muestra número máximo de filas y columnas de data set
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

### Leer los data set del caso de Negocio

In [3]:
# Importamos data y le ponemos formado al ID_CORRELATIVO
dataset = pd.read_csv('train_clientes.csv',sep=';')
dataset['ID_CORRELATIVO'] =   dataset['ID_CORRELATIVO'].apply(lambda x: '{0:0>7}'.format(x))
dataset = dataset.sort_values(['ID_CORRELATIVO'])

In [4]:
#Cabeceras de data
dataset.head()

,ID_CORRELATIVO,CODMES,FLG_BANCARIZADO,RANG_INGRESO,FLAG_LIMA_PROVINCIA,EDAD,ANTIGUEDAD,ATTRITION,RANG_SDO_PASIVO_MENOS0,SDO_ACTIVO_MENOS0,SDO_ACTIVO_MENOS1,SDO_ACTIVO_MENOS2,SDO_ACTIVO_MENOS3,SDO_ACTIVO_MENOS4,SDO_ACTIVO_MENOS5,FLG_SEGURO_MENOS0,FLG_SEGURO_MENOS1,FLG_SEGURO_MENOS2,FLG_SEGURO_MENOS3,FLG_SEGURO_MENOS4,FLG_SEGURO_MENOS5,RANG_NRO_PRODUCTOS_MENOS0,FLG_NOMINA,NRO_ACCES_CANAL1_MENOS0,NRO_ACCES_CANAL1_MENOS1,NRO_ACCES_CANAL1_MENOS2,NRO_ACCES_CANAL1_MENOS3,NRO_ACCES_CANAL1_MENOS4,NRO_ACCES_CANAL1_MENOS5,NRO_ACCES_CANAL2_MENOS0,NRO_ACCES_CANAL2_MENOS1,NRO_ACCES_CANAL2_MENOS2,NRO_ACCES_CANAL2_MENOS3,NRO_ACCES_CANAL2_MENOS4,NRO_ACCES_CANAL2_MENOS5,NRO_ACCES_CANAL3_MENOS0,NRO_ACCES_CANAL3_MENOS1,NRO_ACCES_CANAL3_MENOS2,NRO_ACCES_CANAL3_MENOS3,NRO_ACCES_CANAL3_MENOS4,NRO_ACCES_CANAL3_MENOS5,NRO_ENTID_SSFF_MENOS0,NRO_ENTID_SSFF_MENOS1,NRO_ENTID_SSFF_MENOS2,NRO_ENTID_SSFF_MENOS3,NRO_ENTID_SSFF_MENOS4,NRO_ENTID_SSFF_MENOS5,FLG_SDO_OTSSFF_MENOS0,FLG_SDO_OTSSFF_MENOS1,FLG_SDO_OTSSFF_MENOS2,FLG_SDO_OTSSFF_MENOS3,FLG_SDO_OTSSFF_MENOS4,FLG_SDO_OTSSFF_MENOS5
50778,0000001,201208,1,Rang_ingreso_02,Lima,NaN,0.0,1,Rango_SDO_01,0,0,0,0,0,0,0,0,0,0,0,0,Rango_02,1,1,0,0,0,0,0,0,0,0,0,0,0,9,27,16,26,5,0,0,0,0,0,0,0,0,0,0,0,0,0
68257,0000002,201208,1,Rang_ingreso_01,Provincia,27.0,1.0,0,Rango_SDO_01,0,0,0,0,0,0,1,1,1,1,1,1,Rango_03,0,0,0,0,0,2,1,1,4,3,9,5,0,1,0,0,0,4,0,4,4,4,4,4,4,1,1,1,1,1,1
53623,0000003,201208,0,Rang_ingreso_01,Provincia,34.0,0.0,1,Rango_SDO_01,0,0,0,0,0,0,0,0,0,0,0,0,Rango_02,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
45001,0000004,201208,1,Rang_ingreso_06,Lima,25.0,3.0,0,Rango_SDO_02,0,0,0,0,0,0,1,1,0,0,0,0,Rango_04,1,6,4,1,4,2,1,24,23,17,24,19,16,15,22,25,25,24,17,1,1,0,0,0,0,0,0,0,0,0,0
53741,0000005,201208,0,Rang_ingreso_01,NaN,NaN,0.0,0,Rango_SDO_02,0,0,0,0,0,0,0,0,0,0,0,0,Rango_02,1,0,0,0,0,0,0,0,0,0,0,0,0,15,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
#Muestra las filas y columnas
dataset.shape

(70000, 53)

In [6]:
#Información del data set
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70000 entries, 50778 to 63754
Data columns (total 53 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID_CORRELATIVO             70000 non-null  object 
 1   CODMES                     70000 non-null  int64  
 2   FLG_BANCARIZADO            70000 non-null  int64  
 3   RANG_INGRESO               60583 non-null  object 
 4   FLAG_LIMA_PROVINCIA        66614 non-null  object 
 5   EDAD                       64674 non-null  float64
 6   ANTIGUEDAD                 68238 non-null  float64
 7   ATTRITION                  70000 non-null  int64  
 8   RANG_SDO_PASIVO_MENOS0     70000 non-null  object 
 9   SDO_ACTIVO_MENOS0          70000 non-null  int64  
 10  SDO_ACTIVO_MENOS1          70000 non-null  int64  
 11  SDO_ACTIVO_MENOS2          70000 non-null  int64  
 12  SDO_ACTIVO_MENOS3          70000 non-null  int64  
 13  SDO_ACTIVO_MENOS4          70000 non-null  int6

In [9]:
# Características del data set
print("Number of instances in dataset = {}".format(dataset.shape[0]))
print("Total number of columns = {}".format(dataset.columns.shape[0]))
print("Column wise count of null values:-")
print(dataset.isnull().sum())

Number of instances in dataset = 70000
Total number of columns = 53
Column wise count of null values:-
ID_CORRELATIVO                  0
CODMES                          0
FLG_BANCARIZADO                 0
RANG_INGRESO                 9417
FLAG_LIMA_PROVINCIA          3386
EDAD                         5326
ANTIGUEDAD                   1762
ATTRITION                       0
RANG_SDO_PASIVO_MENOS0          0
SDO_ACTIVO_MENOS0               0
SDO_ACTIVO_MENOS1               0
SDO_ACTIVO_MENOS2               0
SDO_ACTIVO_MENOS3               0
SDO_ACTIVO_MENOS4               0
SDO_ACTIVO_MENOS5               0
FLG_SEGURO_MENOS0               0
FLG_SEGURO_MENOS1               0
FLG_SEGURO_MENOS2               0
FLG_SEGURO_MENOS3               0
FLG_SEGURO_MENOS4               0
FLG_SEGURO_MENOS5               0
RANG_NRO_PRODUCTOS_MENOS0       0
FLG_NOMINA                      0
NRO_ACCES_CANAL1_MENOS0         0
NRO_ACCES_CANAL1_MENOS1         0
NRO_ACCES_CANAL1_MENOS2         0
NRO_ACCES_CAN

### Porcentaje de las clases del Target

In [10]:
#Cantidad y % de categorías del target
print(pd.value_counts(dataset["ATTRITION"]))
print(100 * dataset['ATTRITION'].value_counts() / len(dataset['ATTRITION']))

ATTRITION
0    59178
1    10822
Name: count, dtype: int64
ATTRITION
0    84.54
1    15.46
Name: count, dtype: float64


### Primeras inputaciones

In [11]:
### Imputaciones de valores perdidos
dataset['EDAD'] = dataset['EDAD'].fillna(dataset['EDAD'].mean())
dataset['ANTIGUEDAD'] = dataset['ANTIGUEDAD'].fillna(dataset['ANTIGUEDAD'].mean())
dataset['RANG_INGRESO'] = dataset['RANG_INGRESO'].fillna(dataset['RANG_INGRESO'].mode()[0])
dataset['FLAG_LIMA_PROVINCIA'] = dataset['FLAG_LIMA_PROVINCIA'].fillna(dataset['FLAG_LIMA_PROVINCIA'].mode()[0])

In [12]:
# verificar los data set
dataset.isnull().sum().sum()

0

### Procesamiento de datos

In [13]:
#Variables dummies
dataset = pd.get_dummies(dataset, columns=['RANG_INGRESO','FLAG_LIMA_PROVINCIA','RANG_SDO_PASIVO_MENOS0','RANG_NRO_PRODUCTOS_MENOS0']) 
dataset

,ID_CORRELATIVO,CODMES,FLG_BANCARIZADO,EDAD,ANTIGUEDAD,ATTRITION,SDO_ACTIVO_MENOS0,SDO_ACTIVO_MENOS1,SDO_ACTIVO_MENOS2,SDO_ACTIVO_MENOS3,SDO_ACTIVO_MENOS4,SDO_ACTIVO_MENOS5,FLG_SEGURO_MENOS0,FLG_SEGURO_MENOS1,FLG_SEGURO_MENOS2,FLG_SEGURO_MENOS3,FLG_SEGURO_MENOS4,FLG_SEGURO_MENOS5,FLG_NOMINA,NRO_ACCES_CANAL1_MENOS0,NRO_ACCES_CANAL1_MENOS1,NRO_ACCES_CANAL1_MENOS2,NRO_ACCES_CANAL1_MENOS3,NRO_ACCES_CANAL1_MENOS4,NRO_ACCES_CANAL1_MENOS5,NRO_ACCES_CANAL2_MENOS0,NRO_ACCES_CANAL2_MENOS1,NRO_ACCES_CANAL2_MENOS2,NRO_ACCES_CANAL2_MENOS3,NRO_ACCES_CANAL2_MENOS4,NRO_ACCES_CANAL2_MENOS5,NRO_ACCES_CANAL3_MENOS0,NRO_ACCES_CANAL3_MENOS1,NRO_ACCES_CANAL3_MENOS2,NRO_ACCES_CANAL3_MENOS3,NRO_ACCES_CANAL3_MENOS4,NRO_ACCES_CANAL3_MENOS5,NRO_ENTID_SSFF_MENOS0,NRO_ENTID_SSFF_MENOS1,NRO_ENTID_SSFF_MENOS2,NRO_ENTID_SSFF_MENOS3,NRO_ENTID_SSFF_MENOS4,NRO_ENTID_SSFF_MENOS5,FLG_SDO_OTSSFF_MENOS0,FLG_SDO_OTSSFF_MENOS1,FLG_SDO_OTSSFF_MENOS2,FLG_SDO_OTSSFF_MENOS3,FLG_SDO_OTSSFF_MENOS4,FLG_SDO_OTSSFF_MENOS5,RANG_INGRESO_Rang_ingreso_01,RANG_INGRESO_Rang_ingreso_02,RANG_INGRESO_Rang_ingreso_03,RANG_INGRESO_Rang_ingreso_04,RANG_INGRESO_Rang_ingreso_05,RANG_INGRESO_Rang_ingreso_06,RANG_INGRESO_Rang_ingreso_07,RANG_INGRESO_Rang_ingreso_08,RANG_INGRESO_Rang_ingreso_09,FLAG_LIMA_PROVINCIA_Lima,FLAG_LIMA_PROVINCIA_Provincia,RANG_SDO_PASIVO_MENOS0_Cero,RANG_SDO_PASIVO_MENOS0_Rango_SDO_01,RANG_SDO_PASIVO_MENOS0_Rango_SDO_02,RANG_SDO_PASIVO_MENOS0_Rango_SDO_03,RANG_SDO_PASIVO_MENOS0_Rango_SDO_04,RANG_SDO_PASIVO_MENOS0_Rango_SDO_05,RANG_SDO_PASIVO_MENOS0_Rango_SDO_06,RANG_SDO_PASIVO_MENOS0_Rango_SDO_07,RANG_SDO_PASIVO_MENOS0_Rango_SDO_08,RANG_SDO_PASIVO_MENOS0_Rango_SDO_09,RANG_SDO_PASIVO_MENOS0_Rango_SDO_10,RANG_SDO_PASIVO_MENOS0_Rango_SDO_11,RANG_SDO_PASIVO_MENOS0_Rango_SDO_12,RANG_SDO_PASIVO_MENOS0_Rango_SDO_13,RANG_SDO_PASIVO_MENOS0_Rango_SDO_14,RANG_NRO_PRODUCTOS_MENOS0_Rango_01,RANG_NRO_PRODUCTOS_MENOS0_Rango_02,RANG_NRO_PRODUCTOS_MENOS0_Rango_03,RANG_NRO_PRODUCTOS_MENOS0_Rango_04,RANG_NRO_PRODUCTOS_MENOS0_Rango_05,RANG_NRO_PRODUCTOS_MENOS0_Rango_06
50778,0000001,201208,1,35.232474,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,9,27,16,26,5,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
68257,0000002,201208,1,27.000000,1.0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,2,1,1,4,3,9,5,0,1,0,0,0,4,0,4,4,4,4,4,4,1,1,1,1,1,1,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
53623,0000003,201208,0,34.000000,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
45001,0000004,201208,1,25.000000,3.0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,6,4,1,4,2,1,24,23,17,24,19,16,15,22,25,25,24,17,1,1,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
53741,0000005,201208,0,35.232474,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,15,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33140,0099995,201208,1,30.000000,6.0,0,0,0,0

## Seleccion de variables

In [14]:
#Crección de las matriz X de variables predictoras y el Y el target
X,y = dataset.drop(['ID_CORRELATIVO', 'CODMES','ATTRITION'], axis=1),dataset[["ATTRITION"]]
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=1416,stratify=y)

### Random Forest Features Importance

In [15]:
### Entreno un modelo de Random Forest con 200 arboles y una semilla aleatoria 123
rf_clf = RandomForestClassifier(n_estimators=200,random_state =123)   ## forma del modelo

# Entreno el modelo con el .fit
rf_clf.fit(X_train.values,y_train.values.ravel())
## ordenando  las mejores variables de mayor a menor generando un bucle en este caso un for para
## extraer los nombres de las columnas y la ganancia(Gain) de las variables mas impactantes

features = []
for feature in zip(X.columns, rf_clf.feature_importances_):
    features.append(feature)
    
## genero un dataframe para visualizar mejor lo que hizo random forest con su selector multivariado  
features_total = pd.DataFrame(features,columns=['Variables','Gain']).sort_values('Gain', ascending=False)
features_total

,Variables,Gain
1,EDAD,0.108915
28,NRO_ACCES_CANAL3_MENOS0,0.068042
2,ANTIGUEDAD,0.065140
29,NRO_ACCES_CANAL3_MENOS1,0.063475
58,RANG_SDO_PASIVO_MENOS0_Rango_SDO_01,0.045466
30,NRO_ACCES_CANAL3_MENOS2,0.044848
31,NRO_ACCES_CANAL3_MENOS3,0.033723
34,NRO_ENTID_SSFF_MENOS0,0.024704
46,RANG_INGRESO_Rang_ingreso_01,0.021977
39,NRO_ENTID_SSFF_MENOS5,0.021703


### Selector de Variables

In [16]:
## uso SelectFromModel poniendo lo parametros del modelo que se contruyo y un threshold que es un punto de corte que
## puede asumir de acuerdo a tu criterio con respecto a las variables que estén generando mas Gain

sfm = SelectFromModel(rf_clf, threshold=0.008)

# entrenamiento del selector
sfm.fit(X, y)

# contruyo una lista para quedarme con las mejores variables

variables = [] #### lista vacía
for feature_list_index in sfm.get_support(indices=True): ## 
    variables.append(X.columns[feature_list_index])
    #variables
    
#features_total
variables

['EDAD',
 'ANTIGUEDAD',
 'SDO_ACTIVO_MENOS0',
 'SDO_ACTIVO_MENOS1',
 'FLG_SEGURO_MENOS0',
 'FLG_SEGURO_MENOS1',
 'FLG_NOMINA',
 'NRO_ACCES_CANAL1_MENOS0',
 'NRO_ACCES_CANAL1_MENOS1',
 'NRO_ACCES_CANAL1_MENOS2',
 'NRO_ACCES_CANAL2_MENOS0',
 'NRO_ACCES_CANAL2_MENOS1',
 'NRO_ACCES_CANAL3_MENOS0',
 'NRO_ACCES_CANAL3_MENOS1',
 'NRO_ACCES_CANAL3_MENOS2',
 'NRO_ACCES_CANAL3_MENOS3',
 'NRO_ACCES_CANAL3_MENOS4',
 'NRO_ACCES_CANAL3_MENOS5',
 'NRO_ENTID_SSFF_MENOS0',
 'NRO_ENTID_SSFF_MENOS1',
 'NRO_ENTID_SSFF_MENOS2',
 'NRO_ENTID_SSFF_MENOS3',
 'NRO_ENTID_SSFF_MENOS4',
 'NRO_ENTID_SSFF_MENOS5',
 'FLG_SDO_OTSSFF_MENOS0',
 'RANG_INGRESO_Rang_ingreso_01',
 'RANG_INGRESO_Rang_ingreso_02',
 'RANG_INGRESO_Rang_ingreso_03',
 'FLAG_LIMA_PROVINCIA_Lima',
 'FLAG_LIMA_PROVINCIA_Provincia',
 'RANG_SDO_PASIVO_MENOS0_Cero',
 'RANG_SDO_PASIVO_MENOS0_Rango_SDO_01',
 'RANG_NRO_PRODUCTOS_MENOS0_Rango_01',
 'RANG_NRO_PRODUCTOS_MENOS0_Rango_02']

## Construcción de modelos de Machine Learning

In [17]:
## Usando la mejor selección de variables
best_X_train = X_train[variables]
best_X_test  = X_test[variables]

In [18]:
best_X_train.shape

(56000, 34)

In [19]:
from time import time

# construimos un lightgbm, ojo que para elY es el mismo porque es un vector univariado
best_model = LGBMClassifier() 
best_model.fit(best_X_train, y_train)


# predecimos el train y test con la probabilidad para validar luego
predict_train_lg = best_model.predict_proba(best_X_train)[:,1]
predict_test_lg = best_model.predict_proba(best_X_test)[:,1]


# imprimimos el roc de train y test con la data real y la prediccion de la probabilidad del modelo
print("auc o Roc on training in LGBMClassifier data : {:.3f}".format(roc_auc_score(y_train, predict_train_lg)))
print("auc o Roc on testing in LGBMClassifier  data : {:.3f}".format(roc_auc_score(y_test, predict_test_lg))) 

[LightGBM] [Info] Number of positive: 8658, number of negative: 47342
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015670 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1282
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.154607 -> initscore=-1.698914
[LightGBM] [Info] Start training from score -1.698914
auc o Roc on training in LGBMClassifier data : 0.892
auc o Roc on testing in LGBMClassifier  data : 0.851


## Cross Validation and Optimización de Modelos 

In [20]:
from pprint import pprint
# Mostramos todos los parametros que tiene ligthGBMlo usamos porque es mas rápido para tunear el modelo
print('Parameters currently in use:\n')
pprint(best_model.get_params()) 

Parameters currently in use:

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}


## Grid Search with Cross Validation

In [21]:
# Numero de arboles
n_estimators = [300, 400, 500]
# porcentaje de variables con la que se contruye un arbol
colsample_bytree = [0.7, 0.8, 0.9]
# profundidad del arbol
max_depth = [4, 6, 8]
# ratio de aprendizaje por cada arbol
learning_rate = [0.1, 0.15 ,0.2]
# creacion del grip search o grilla generando un diccionario
param_grid = {'n_estimators': n_estimators,
               'colsample_bytree': colsample_bytree,
               'max_depth': max_depth,
               'learning_rate': learning_rate}
pprint(param_grid)

{'colsample_bytree': [0.7, 0.8, 0.9],
 'learning_rate': [0.1, 0.15, 0.2],
 'max_depth': [4, 6, 8],
 'n_estimators': [300, 400, 500]}


In [22]:
lgb = LGBMClassifier()
# Isntacia del grip search ponemos la grilla , la cantidad de kfolds para la validacion cruzada en este caso 5
# n_jobs -1 para la paralelizacion de la ejecucion en la optimizacion del modelo y verbose para mostrar de 2 en 2 los resultados
grid_search = GridSearchCV(estimator = lgb, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)
# FIT para ejecutarlo
X_best = X[variables] ## recordar que se usa toda la data completa y con las mejores variables
grid_search.fit(X_best, y)
grid_search.best_params_ # mostramos los mejores hiperparametros

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 10822, number of negative: 59178
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011744 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1307
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.154600 -> initscore=-1.698969
[LightGBM] [Info] Start training from score -1.698969
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

{'colsample_bytree': 0.7,
 'learning_rate': 0.1,
 'max_depth': 8,
 'n_estimators': 500}

### Evaluacion 

In [23]:
# Genero una funcion para validar la mejora de la curva ROC o auc para ver como aumenta el performance de mi modelo
def evaluate(model, test_features, test_labels):
    predictions = model.predict_proba(test_features)[:,1] ## recordar que para validar el roc se necesita el dato real y 
    roc = roc_auc_score(test_labels,predictions)          ## probabilidad de la prediccion
    print('Model Performance')
    print('roc_auc_score = {:.12g}%.'.format(roc))
    return roc

In [24]:
base_model = LGBMClassifier()
base_model.fit(best_X_train, y_train)
base_roc = evaluate(base_model, best_X_test, y_test)

## pasamos la funcion evaluate para sacar el resultado del modelo
best_grid = grid_search.best_estimator_
grid_roc = evaluate(best_grid, best_X_test, y_test)
print('Mejora en {:.12g}%.'.format( 100 * (grid_roc - base_roc) / base_roc)) ## le damos diseño para imprimir con .format
## podemos visualizar el aumento o las mejora en 3.5%

[LightGBM] [Info] Number of positive: 8658, number of negative: 47342
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004644 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1282
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.154607 -> initscore=-1.698914
[LightGBM] [Info] Start training from score -1.698914
Model Performance
roc_auc_score = 0.851287528446%.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Model Performance
roc_auc_score = 0.936781754371%.
Mejora en 10.0429318024%.


In [25]:
base_roc ## roc para el test  ---- modelo normal

0.8512875284463766

In [26]:
grid_roc ## roc para el test --- modelo optimizado

0.9367817543707315

In [27]:
base_model = LGBMClassifier()
base_model.fit(best_X_train, y_train)
base_roc = evaluate(base_model, best_X_train, y_train)

## pasamos la funcion evaluate para sacar el resultado del modelo
best_grid = grid_search.best_estimator_
grid_roc = evaluate(best_grid, best_X_train, y_train)
print('Mejora en {:.12g}%.'.format( 100 * (grid_roc - base_roc) / base_roc)) ## le damos diseño para imprimir con .format
## podemos visualizar el aumento o las mejora en 3.5%

[LightGBM] [Info] Number of positive: 8658, number of negative: 47342
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013008 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1282
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.154607 -> initscore=-1.698914
[LightGBM] [Info] Start training from score -1.698914
Model Performance
roc_auc_score = 0.891517382609%.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Model Performance
roc_auc_score = 0.93440402443%.
Mejora en 4.81052222397%.
